# #6246 Initialize should be composed of Resets + StatePreparation

### Motivation

- Existing `Initialize` class adds a reset followed by some state preparation functionality (differs for int, string, statevector etc.)
- Users want to be able to prepare state without automatically adding a reset, use more `Gate` functions (`Initialize` is an Instruction, can't inverse, add controls etc.)
- Original GitHub issue:https://github.com/Qiskit/qiskit-terra/issues/6246
- Stack Exchange: https://quantumcomputing.stackexchange.com/questions/24535/is-it-possible-to-create-a-controlled-initialize-instruction-in-qiskit

### New StatePreparation Class

TL;DR 

a `Gate` that does everything `Initialize` used to do, without adding a reset

`Initialize` now adds a reset calls `StatePreparation` under the hood

Users can continue to use `Initialize` the same as before

### Imports

In [64]:
import numpy as np
import math
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

from qiskit.extensions.quantum_initializer import Initialize
from qiskit.circuit.library import StatePreparation

### Initialize Implementation

In [17]:
qr = QuantumRegister(2)
qc_init = QuantumCircuit(qr)

# int param
init = Initialize(2)

# string param
# init = Initialize('11')

# statevector param
# init = Initialize([1 / math.sqrt(2), 0, 0, 1 / math.sqrt(2)])

qc_init.append(init, qr)

qc_init.draw()

┌──────────────────────────────────┐
q8_0: ┤0                                 ├
      │  Initialize(0.70711,0,0,0.70711) │
q8_1: ┤1                                 ├
      └──────────────────────────────────┘

In [18]:
qc_init.decompose().draw()

┌─────────────────────────────────────────┐
q8_0: ─|0>─┤0                                        ├
           │  State Preparation(0.70711,0,0,0.70711) │
q8_1: ─|0>─┤1                                        ├
           └─────────────────────────────────────────┘

In [19]:
qc_init.decompose().decompose().draw()

┌──────────────────┐
q8_0: ─|0>─┤0                 ├
           │  disentangler_dg │
q8_1: ─|0>─┤1                 ├
           └──────────────────┘

In [21]:
qc_init2 = QuantumCircuit(2)

qc_init2.initialize(2)

qc_init2.draw()

┌────────────────┐
q_0: ┤0               ├
     │  Initialize(2) │
q_1: ┤1               ├
     └────────────────┘

### StatePreparation Class Implementation

In [53]:
circ = QuantumCircuit(2)

# int param
# my_state = StatePreparation(2, label="my state")

# string param
# my_state = StatePreparation('11', label="my state")

# statevector param
# my_state = StatePreparation([1 / math.sqrt(2), 0, 0, 1 / math.sqrt(2)], label="my state")

circ.append(my_state, [0,1])

circ.draw()

┌────────────────────────────────┐
q_0: ┤0                               ├
     │  my state(0.70711,0,0,0.70711) │
q_1: ┤1                               ├
     └────────────────────────────────┘

In [54]:
circ.decompose().draw()

┌──────────────────┐
q_0: ┤0                 ├
     │  disentangler_dg │
q_1: ┤1                 ├
     └──────────────────┘

### StatePreparation inverse

In [55]:
circ.append(my_state.inverse(), [0,1])
circ.draw()

┌────────────────────────────────┐┌───────────────────────────────────┐
q_0: ┤0                               ├┤0                                  ├
     │  my state(0.70711,0,0,0.70711) ││  my state Dg(0.70711,0,0,0.70711) │
q_1: ┤1                               ├┤1                                  ├
     └────────────────────────────────┘└───────────────────────────────────┘

In [56]:
circ.decompose().draw()

┌──────────────────┐┌───────────────┐
q_0: ┤0                 ├┤0              ├
     │  disentangler_dg ││  disentangler │
q_1: ┤1                 ├┤1              ├
     └──────────────────┘└───────────────┘

### StatePreparation Control

In [72]:
qc_control = QuantumCircuit(3)

stateprep = StatePreparation('11')
controlled_stateprep = stateprep.control(1)

qc_control.append(controlled_stateprep, [0,1,2])

qc_control.draw()

q_0: ─────────────■─────────────
     ┌────────────┴────────────┐
q_1: ┤0                        ├
     │  State Preparation(1,1) │
q_2: ┤1                        ├
     └─────────────────────────┘

### prepare_state() a la initialize()

In [58]:
qc = QuantumCircuit(2)

# int param
# qc.prepare_state(2, label="my cool state")

# string param
# qc.prepare_state('11', label="my cool state")

# statevector param
qc.prepare_state([1 / math.sqrt(2), 0, 0, 1 / math.sqrt(2)], label="my cool state")

qc.draw()

┌─────────────────────────────────────┐
q_0: ┤0                                    ├
     │  my cool state(0.70711,0,0,0.70711) │
q_1: ┤1                                    ├
     └─────────────────────────────────────┘

## Potential Areas of Further Development

- Could allow users to specify synthesis (e.g. could call `Isometry`)
- Currently `StatePreparation` assumes starting in |0> state